In [3]:

import numpy as np
from scipy.optimize import fsolve

In [4]:

# Constants
c_p = 1.013  # kJ/kg/K, specific heat of dry air at constant pressure
R_v = 0.4615  # kJ/kg/K, specific gas constant for water vapor
R_d = 0.287  # kJ/kg/K, specific gas constant for dry air
K = 273.15  # Conversion factor from Celsius to Kelvin

def calc_esat(T):
    """
    Calculate saturation vapor pressure from temperature
    T: temperature in degrees Celsius
    return: saturation vapor pressure in kPa
    """
    esat = 0.6108 * np.exp(17.27 * T / (T + 237.3)) # Eq. 2.17 TH
    return esat

def calc_e(T, RH=None, T_dew=None, rho_v=None, mode="RH"):
    """
    Calculate vapor pressure from relative humidity, dewpoint temperature, or vapor density
    T: temperature in degrees Celsius
    RH: Relative humidity in percent
    T_dew: Dewpoint temperature in degrees Celsius
    rho_v: Vapor density in kg/m^3
    mode: Method to calculate vapor pressure ('RH', 'T_dew', 'rho_v')
    return: vapor pressure in kPa
    """
    if mode == "RH" and RH is not None: 
        e = RH * calc_esat(T)
    elif mode == "T_dew" and T_dew is not None:
        e = 0.6108 * np.exp(17.27 * T_dew / (T_dew + 237.3))
    elif mode == "rho_v" and rho_v is not None:
        e = rho_v * R_v * (T + K)
    else:
        e = None
    return e

def calc_delta(T):
    """
    Calculate gradient of the saturation vapor pressure curve from saturation vapor pressure and temperature
    e_sat: saturation vapor pressure in kPa
    T: temperature in degrees Celsius
    return: slope of the saturation vapor pressure curve in kPa/°C
    """
    e_sat = calc_esat(T)
    delta = 4098 * e_sat / (T + 237.3)**2 # Eq 2.18 TH
    return delta

def calc_LH(T):
    """
    Calculate latent heat of vaporization from temperature
    T: temperature in degrees Celsius
    return: latent heat of vaporization (lambda) in MJ/kg
    """
    LH = 2.501 - 0.002361 * T # Eq 2.1 TH
    return LH

def calc_psy_const(P, LH):
    """
    Calculate psychrometric constant
    c_p: specific heat of dry air at constant pressure in kJ/kg/K
    P: pressure in kPa
    LH: latent heat of vaporization in MJ/kg
    return: psychrometric constant in kPa/°C
    """
    psychrometric_constant = c_p * P / (0.622 * LH) # Eq. 2.24 TH
    return psychrometric_constant

def calc_rho_d(P, T):
    """
    Calculate dry air density from pressure and temperature
    P: pressure in kPa
    T: temperature in degrees Celsius
    return: dry air density in kg/m^3
    """
    rho_d = P / (R_d * (T + K))
    return rho_d

def calc_rho_v(e, T):
    """
    Calculate vapor density from vapor pressure and temperature
    e: vapor pressure in kPa
    T: temperature in degrees Celsius
    return: vapor density in kg/m^3
    """
    rho_v = e / (R_v * (T + K))
    return rho_v

def calc_rho(rho_d, rho_v):
    """
    Calculate air density from dry air density and vapor density
    rho_d: dry air density in kg/m^3
    rho_v: vapor density in kg/m^3
    return: air density in kg/m^3
    """
    rho = rho_d + rho_v
    return rho

def calc_RH(e, e_sat):
    """
    Calculate relative humidity from vapor pressure and saturation vapor pressure
    e: vapor pressure in kPa
    e_sat: saturation vapor pressure in kPa
    return: relative humidity in percent
    """
    RH = (e / e_sat) * 100  # Adjusted to return percentage
    return RH

def calc_dewpoint(e):
    """
    Calculate dewpoint temperature (T_dew) from vapor pressure
    e: vapor pressure in kPa
    return: dewpoint temperature in degrees Celsius
    """
    T_dew = (np.log(e) + 0.49299) / (0.0707 - 0.00421 * np.log(e))
    return T_dew

def calc_specific_humidity(rho_v, rho_d):
    """
    Calculate specific humidity from vapor density and dry air density in moist air sample
    rho_v: vapor density in kg/m^3
    rho_d: dry air density in kg/m^3
    return: specific humidity in kg/kg
    """
    q = rho_v / (rho_v + rho_d)  # Adjusted units to kg/kg for consistency
    return q

def calc_mixing_ratio(rho_v, rho_d):
    """
    Calculate mixing ratio from vapor density and dry air density in moist air sample
    rho_v: vapor density in kg/m^3
    rho_d: dry air density in kg/m^3
    return: mixing ratio in kg/kg
    """
    r = rho_v / rho_d
    return r

def calc_virtual_temp(T, mixing_ratio):
    """
    Calculate virtual temperature from temperature and mixing ratio
    T: temperature in degrees Celsius
    mixing_ratio: mixing ratio in kg/kg
    return: virtual temperature in degrees Celsius
    """
    T_v = T * (1 + 0.61 * mixing_ratio) # Eq 2.13 TH
    return T_v

def calc_wetbulb(T, e, P = None, LH = None, psy = None):
    """
    Calculate wetbulb temperature (T_wet) from temperature, vapor pressure, pressure, and latent heat of vaporization
    T: dry bulb temperature in degrees Celsius
    e: vapor pressure in kPa
    P: pressure in kPa
    LH: latent heat of vaporization in MJ/kg
    return: wetbulb temperature in degrees Celsius
    """
    if psy is None:
        psy = calc_psy_const(P, LH)

    def equation_to_solve(T_wet):
        return e - calc_esat(T_wet) + psy * (T - T_wet)

    T_wet_init = T * 0.85  # Initial guess for T_wet, assuming it's lower than dry bulb temperature
    T_wet_solution = fsolve(equation_to_solve, T_wet_init)
    return T_wet_solution[0]

In [9]:
T = 32 # Dry bulb temperature in degrees Celsius
RH = 0.50 # Relative humidity in percent
P = 95 # Atmospheric pressure in kPa
psy = 0.066 # Psychrometric constant in kPa/°C

e = calc_e(T, RH=RH) # Vapor pressure in kPa
e_sat = calc_esat(T) # Saturation vapor pressure in kPa
LH = calc_LH(T) # Latent heat of vaporization in MJ/kg

T_dew = calc_dewpoint(e) # Dew point temperature in degrees Celsius
T_wet = calc_wetbulb(T, e, psy = psy) # Wet bulb temperature in degrees Celsius
q = calc_specific_humidity(calc_rho_v(e, T), calc_rho_d(P, T)) # Specific humidity in kg/kg
mixing_ratio = calc_mixing_ratio(calc_rho_v(e, T), calc_rho_d(P, T)) # Mixing ratio in kg/kg

print(f"Temperature: {T}°C")
print(f"Relative Humidity: {RH*100}%")
print(f"Vapor Pressure: {e:.2f} kPa")
print(f"Saturation Vapor Pressure: {e_sat:.2f} kPa\n")
print("Problem 1 Answers:")
print(f"a. Dew Point: {T_dew:.2f}°C")
print(f"b. Wet Bulb: {T_wet:.2f}°C")
print(f"c. Specific Humidity: {q:.4f} kg/kg")
print(f"d. Mixing Ratio: {mixing_ratio:.4f} kg/kg")

Temperature: 32°C
Relative Humidity: 50.0%
Vapor Pressure: 2.38 kPa
Saturation Vapor Pressure: 4.75 kPa

Problem 1 Answers:
a. Dew Point: 20.27°C
b. Wet Bulb: 23.68°C
c. Specific Humidity: 0.0153 kg/kg
d. Mixing Ratio: 0.0156 kg/kg


In [6]:
sigma = 5.67 * (10**-8)  # Stefan-Boltzmann constant in W/m^2/K^4
Kelvin = 273.15  # Conversion factor from Celsius to Kelvin

def calc_S_net(S_in, albedo):
    """
    Calculate net short wave radiation from incident short wave solar radiation and albedo.
    S_in: Incident short wave solar radiation in W/m^2.
    albedo: Albedo of the surface.
    return: Net short wave radiation in W/m^2.
    """
    S_net = S_in * (1 - albedo)
    return S_net

def calc_LW(T_s, emissivity):
    """
    Calculate long wave radiation from surface temperature and emissivity.
    T_s: Surface temperature in degrees Celsius.
    emissivity: Emissivity of the surface.
    return: Long wave radiation in W/m^2.
    """
    LW = emissivity * sigma * (T_s + Kelvin)**4
    return LW

def calc_LW_net(T_a, e, f=1):
    """
    Calculate empirical net long wave radiation from air temperature, vapor pressure, and cloud factor.
    T_a: Air temperature in degrees Celsius.
    e: Vapor pressure in kPa.
    f: Cloud factor (1 = cloudless sky).
    return: Net long wave radiation in W/m^2.
    """
    eff_emissivity = 0.34 - 0.14 * np.sqrt(e) # Eq. 5.23 TH
    LW_net = f * eff_emissivity * sigma * (T_a + Kelvin)**4  # Eq. 5.22 TH
    return LW_net

def calc_R_n(S_net, albedo, LW_net):
    """
    Calculate net radiation from incident short wave solar radiation, albedo, and net long wave radiation.
    S_net: Net short wave radiation in W/m^2.
    albedo: Albedo of the surface.
    LW_net: Net long wave radiation in W/m^2.
    return: Net radiation in W/m^2.
    """
    R_n = S_net * (1 - albedo) + LW_net # Eq. 5.28 in TH
    return R_n

def calc_G(R_n, H, E):
    """
    Calculate heat flux into the ground from net radiation, turbulent sensible heat flux, and evaporation rate.
    R_n: Net radiation in W/m^2.
    H: Turbulent sensible heat flux in W/m^2.
    E: Evaporation rate in m/s.
    return: Heat flux into the ground in W/m^2.
    """
    G = R_n - H - E
    return G

def calc_E(R_n, H, G):
    """
    Calculate evaporation rate from net radiation, turbulent sensible heat flux, and heat flux into the ground.
    R_n: Net radiation in W/m^2.
    H: Turbulent sensible heat flux in W/m^2.
    G: Heat flux into the ground in W/m^2.
    return: Evaporation rate in m/s.
    """
    E = R_n - H - G
    return E

def calc_H(R_n, E, G):
    """
    Calculate turbulent sensible heat flux from net radiation, evaporation rate, and heat flux into the ground.
    R_n: Net radiation in W/m^2.
    E: Evaporation rate in m/s.
    G: Heat flux into the ground in W/m^2.
    return: Turbulent sensible heat flux in W/m^2.
    """
    H = R_n - E - G
    return H


In [11]:

T_aC = 17.94 # Air temperature in degrees Celsius
T_a = T_aC + Kelvin # Air temperature in degrees Kelvin
T_s = T_a  # Surface temperature in degrees Kelvin
RH = 0.66 # Relative humidity in percent
S_in = 468 # Incident short wave solar radiation in cal / cm^2 / day
albedo = 0.23 # From Table 5.2 Albedo for short grass chosen from middle of range

day_to_sec = 24*60*60 # Conversion factor from days to seconds
cal_to_J = 4.184 # Conversion factor from cal to W
cm2_to_m2 = 1 / 10**-4 # Conversion factor from cm^2 to m^2

e = calc_e(T_aC, RH=RH)  # Vapor pressure in kPa

S_in = 468 * cal_to_J * cm2_to_m2 / day_to_sec # Conversion factor from cal/cm^2/day to W/m^2
S_net = calc_S_net(S_in, albedo)
LW_net = calc_LW_net(T_a, e)
R_n = calc_R_n(S_net, albedo, LW_net)

print(f"Vapor pressure: {e:.2f} kPa")
print(f"Net Short Wave Radiation: {S_net:.2f} W/m^2")  # Expected output: 360.00 W/m^2
print(f"Net Long Wave Radiation: {LW_net:.2f} W/m^2\n")  # Expected output: 100.00 W/m^2
print("Problem 4 Answers:")
print(f"Net Radiation: {R_n:.2f} W/m^2")  # Expected output: 100.00 W/m^2

Vapor pressure: 1.36 kPa
Net Short Wave Radiation: 174.51 W/m^2
Net Long Wave Radiation: 1016.68 W/m^2

Problem 4 Answers:
Net Radiation: 1151.05 W/m^2


In [12]:
R_n = 200 # Net radiation in W/m^2
G = 40 # Heat flux into the ground in W/m^2

E = 5 * 10**-8 # Evaporation rate in m/s
E = E * 1000 * (24 * 60 * 60)  # Conversion factor from m/s to mm/day
E = E * 28.6 # 1 mm/day is equivalent to 28.6 W m−2

H = calc_H(R_n, E, G)

print("Problem 5 Answers:")
print(f"Latent Heat Flux: {E:.2f} W/m^2")  
print(f"Sensible Heat Flux: {H:.2f} W/m^2")  

Problem 5 Answers:
Latent Heat Flux: 123.55 W/m^2
Sensible Heat Flux: 36.45 W/m^2
